#  Course on Introduction to Power Systems
Professor: David Pozo
Skoltech, Feb-Mar 2018

## Simple AC-OPF: 3-bus example


In [1]:
using JuMP
using Ipopt

###  Data definition

We define data fo our problem

In [2]:
# Ybus
zL = 0.01 + 0.1im;
Ybus = ones(3,3)*(-1/zL);  # off-diagonal elements
Ybus[1,1] = 2*(1/zL);  # diagonal elements
Ybus[2,2] = 2*(1/zL);  # diagonal elements
Ybus[3,3] = 2*(1/zL);  # diagonal elements

Gbus = real(Ybus);
Bbus = imag(Ybus);

In [3]:
# Generator limits
Pmin = [0,0];
Pmax = [3, 0.8];
Qmin = [-2, -2];
Qmax = [2, 2];

# Generation costs
c = [2,1];

In [4]:
# Line limits
SLmax = [0.25, 2, 2];


In [5]:
# Voltage limits
Vmin = 0.95;
Vmax = 1.10;

In [6]:
# Demand
Pd = 0.8;
Qd = 0.6;

### Model definition

Firstly, we create a model "m" that will gather the main information (decision variables, constraints and objective) of the presented AC-OPF formulation.

In [7]:
m = Model(solver = IpoptSolver());

In [8]:
@variable(m, p[1:2]);
@variable(m, q[1:2]);
@variable(m, pl[1:3,1:3]);
@variable(m, ql[1:3,1:3]);
@variable(m, v[1:3]);
@variable(m, θ[1:3]);

In [9]:
# Nodal balance equation
@constraint(m, p[1]  == pl[1,2] + pl[1,3]);
@constraint(m, p[2]  == pl[2,1] + pl[2,3]);
@constraint(m, - Pd  == pl[3,1] + pl[3,2]);

@constraint(m, q[1]  == ql[1,2] + ql[1,3]);
@constraint(m, q[2]  == ql[2,1] + ql[2,3]);
@constraint(m, - Qd  == ql[3,1] + ql[3,2]);

In [10]:
# Branch power flow definition
@NLconstraint(m, pl[1,2] == -(v[1]^2)*Gbus[1,2] + v[1]*v[2]*(Gbus[1,2]*cos(θ[1]-θ[2]) +  Bbus[1,2]*sin(θ[1]-θ[2])));
@NLconstraint(m, pl[1,3] == -(v[1]^2)*Gbus[1,3] + v[1]*v[3]*(Gbus[1,3]*cos(θ[1]-θ[3]) +  Bbus[1,3]*sin(θ[1]-θ[3])));
@NLconstraint(m, pl[2,1] == -(v[2]^2)*Gbus[2,1] + v[2]*v[1]*(Gbus[2,1]*cos(θ[2]-θ[1]) +  Bbus[2,1]*sin(θ[2]-θ[1])));
@NLconstraint(m, pl[2,3] == -(v[2]^2)*Gbus[2,3] + v[2]*v[3]*(Gbus[2,3]*cos(θ[2]-θ[3]) +  Bbus[2,3]*sin(θ[2]-θ[3])));
@NLconstraint(m, pl[3,1] == -(v[3]^2)*Gbus[3,1] + v[3]*v[1]*(Gbus[3,1]*cos(θ[3]-θ[1]) +  Bbus[3,1]*sin(θ[3]-θ[1])));
@NLconstraint(m, pl[3,2] == -(v[3]^2)*Gbus[3,2] + v[3]*v[2]*(Gbus[3,2]*cos(θ[3]-θ[2]) +  Bbus[3,2]*sin(θ[3]-θ[2])));

@NLconstraint(m, ql[1,2] == (v[1]^2)*Bbus[1,2] + v[1]*v[2]*(Gbus[1,2]*sin(θ[1]-θ[2]) -  Bbus[1,2]*cos(θ[1]-θ[2])));
@NLconstraint(m, ql[1,3] == (v[1]^2)*Bbus[1,3] + v[1]*v[3]*(Gbus[1,3]*sin(θ[1]-θ[3]) -  Bbus[1,3]*cos(θ[1]-θ[3])));
@NLconstraint(m, ql[2,1] == (v[2]^2)*Bbus[2,1] + v[2]*v[1]*(Gbus[2,1]*sin(θ[2]-θ[1]) -  Bbus[2,1]*cos(θ[2]-θ[1])));
@NLconstraint(m, ql[2,3] == (v[2]^2)*Bbus[2,3] + v[2]*v[3]*(Gbus[2,3]*sin(θ[2]-θ[3]) -  Bbus[2,3]*cos(θ[2]-θ[3])));
@NLconstraint(m, ql[3,1] == (v[3]^2)*Bbus[3,1] + v[3]*v[1]*(Gbus[3,1]*sin(θ[3]-θ[1]) -  Bbus[3,1]*cos(θ[3]-θ[1])));
@NLconstraint(m, ql[3,2] == (v[3]^2)*Bbus[3,2] + v[3]*v[2]*(Gbus[3,2]*sin(θ[3]-θ[2]) -  Bbus[3,2]*cos(θ[3]-θ[2])));

In [11]:
# Technical generation limits
@constraint(m, Pmin[1] <= p[1] <= Pmax[1]);
@constraint(m, Pmin[2] <= p[2] <= Pmax[2]);
@constraint(m, Qmin[1] <= q[1] <= Qmax[1]);
@constraint(m, Qmin[1] <= q[2] <= Qmax[2]);

In [12]:
# Voltage limits
@constraint(m, Vmin <= v[1] <= Vmax);
@constraint(m, Vmin <= v[2] <= Vmax);
@constraint(m, Vmin <= v[3] <= Vmax);

# Angle limits
@constraint(m,  θ[1] == 0);
@constraint(m, -pi <= θ[2] <= pi);
@constraint(m, -pi <= θ[3] <= pi);

In [13]:
# Line capacity limits
@NLconstraint(m,  (pl[1,2])^2 + (ql[1,2])^2 <= (SLmax[1])^2);
@NLconstraint(m,  (pl[2,1])^2 + (ql[2,1])^2 <= (SLmax[1])^2);
@NLconstraint(m,  (pl[1,3])^2 + (ql[1,3])^2 <= (SLmax[2])^2);
@NLconstraint(m,  (pl[3,1])^2 + (ql[3,1])^2 <= (SLmax[2])^2);
@NLconstraint(m,  (pl[2,3])^2 + (ql[2,3])^2 <= (SLmax[3])^2);
@NLconstraint(m,  (pl[3,2])^2 + (ql[3,2])^2 <= (SLmax[3])^2);

### Objective function

In [14]:
## Objective function
@objective(m, Min, c[1]*p[1] + c[2]*p[2]);

In [15]:
# Print the model
print(m)

Min 2 p[1] + p[2]
Subject to
 p[1] - pl[1,2] - pl[1,3] = 0
 p[2] - pl[2,1] - pl[2,3] = 0
 -pl[3,1] - pl[3,2] = 0.8
 q[1] - ql[1,2] - ql[1,3] = 0
 q[2] - ql[2,1] - ql[2,3] = 0
 -ql[3,1] - ql[3,2] = 0.6
 0 ≤ p[1] ≤ 3
 0 ≤ p[2] ≤ 0.8
 -2 ≤ q[1] ≤ 2
 -2 ≤ q[2] ≤ 2
 0.95 ≤ v[1] ≤ 1.1
 0.95 ≤ v[2] ≤ 1.1
 0.95 ≤ v[3] ≤ 1.1
 θ[1] = 0
 -3.141592653589793 ≤ θ[2] ≤ 3.141592653589793
 -3.141592653589793 ≤ θ[3] ≤ 3.141592653589793
 pl[1,2] - (-(v[1] ^ 2.0) * -0.99009900990099 + v[1] * v[2] * (-0.99009900990099 * cos(θ[1] - θ[2]) + 9.900990099009901 * sin(θ[1] - θ[2]))) = 0
 pl[1,3] - (-(v[1] ^ 2.0) * -0.99009900990099 + v[1] * v[3] * (-0.99009900990099 * cos(θ[1] - θ[3]) + 9.900990099009901 * sin(θ[1] - θ[3]))) = 0
 pl[2,1] - (-(v[2] ^ 2.0) * -0.99009900990099 + v[2] * v[1] * (-0.99009900990099 * cos(θ[2] - θ[1]) + 9.900990099009901 * sin(θ[2] - θ[1]))) = 0
 pl[2,3] - (-(v[2] ^ 2.0) * -0.99009900990099 + v[2] * v[3] * (-0.99009900990099 * cos(θ[2] - θ[3]) + 9.900990099009901 * sin(θ[2] - θ[3]))) = 

In [16]:
solve(m)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       77
Number of nonzeros in inequality constraint Jacobian.:       21
Number of nonzeros in Lagrangian Hessian.............:      132

Total number of variables............................:       28
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [17]:
getvalue(p)

2-element Array{Float64,1}:
 0.0309068
 0.774265 

In [18]:
getvalue(v)

3-element Array{Float64,1}:
 1.0977 
 1.1    
 1.06636

In [19]:
getvalue(pl)

3×3 Array{Float64,2}:
  0.0       -0.249467  0.280374
  0.249983   0.0       0.524282
 -0.278879  -0.521121  0.0     